In [6]:
%cd "/home/user/Desktop/elmo/anlp-assgn2-data/"
%ls

/home/user/Desktop/elmo/anlp-assgn2-data
ELMO_scratch.ipynb  readme.txt           yelp-subset.test.csv
glove.txt           yelp-subset.dev.csv  yelp-subset.train.csv


In [7]:
#imports  
import pandas as pd
import re
import torch#
from torch import nn, optim
import torch.nn.functional as F

In [8]:
train = pd.read_csv('yelp-subset.train.csv', sep='\t', header=None)
test = pd.read_csv('yelp-subset.test.csv', sep='\t', header=None)


train['label'] = train[0].apply(lambda x: x.split(',')[0])
train['text'] = train[0].apply(lambda x: x.split(',', 1)[1])
# removes 0 col 
train = train.drop(columns=[0])
train = train.iloc[1: , :]
test['label'] = test[0].apply(lambda x: x.split(',')[0])
test['text'] = test[0].apply(lambda x: x.split(',', 1)[1])
test = test.drop(columns=[0])
test = test.iloc[1: , :]

data = pd.concat([train, test])

#cleaning the data
print(data.shape)

data['text'] = data['text'].apply(lambda x: re.sub(r'http\S+', '', x))


punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

data['text'] = data['text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))


# convert text to lowercase
data['text'] = data['text'].str.lower()


# remove numbers
data['text'] = data['text'].str.replace("[0-9]", " ")


# remove whitespaces
data['text'] = data['text'].apply(lambda x:' '.join(x.split()))

(55000, 2)


/tmp/ipykernel_90554/1588324648.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace("[0-9]", " ")


In [9]:
# truncate the dataset
new_data = data.iloc[:1000]

In [10]:
# print one sentence from the dataset 'text'
# for every sentence in the text on data set, extract the sentence based on full stop
# and store each sentence in a sentences dataframe                                                                                                                                                                                                                                                                                                      
sentences = []

for s in new_data['text']:
    sentences.append(s.split('.'))


# flatten the list of lists
sentences = [y for x in sentences for y in x]
print(len(sentences))

# remove empty sentences
sentences = [s for s in sentences if len(s) > 0]
print(len(sentences))

# for every sentence in the sentences list, attach <eos> and <sos> tags
sentences = ['<sos> ' + s + ' <eos>' for s in sentences]


new_sentences= []
words = []
for s in sentences:
    new_sentences.append(s.split())

print(len(new_sentences))
sentences = new_sentences
print(sentences[10])

11308
9224
9224
['<sos>', 'of', 'course,', 'at', "mike's", 'you', 'would', 'probably', 'be', 'stampeded,', 'but', "that's", 'another', 'issue', 'entirely', '<eos>']


In [11]:
#creating a index od each word. ex: she , he padd word[1]= she or word[2]= he...we printing vocab length at the end.
vocb = {}
for sentence in sentences:
    for word in sentence:
        if word not in vocb:
            vocb[word] = 1
        else:
            vocb[word] += 1


word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}
print(len(vocb))


12594


In [12]:
#creats embedding dictionary. it is a dictionary of words and their corresponding vectors.
import numpy as np
embeddings_dict = {}
word_dict={}
with open("glove.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [13]:

import random

etag = random.randint(1,10)
stag = random.randint(1,10)

eos_tag = [etag for i in range(50)]
sos_tag = [stag for i in range(50)]

# word and corrsponding vector
def find_embedding(word):
  if(word=="<sos>"):
    return sos_tag
  elif(word=="<eos>"):
    return eos_tag

  vector=[0 for i in range(50)]
  try:
    vector=embeddings_dict[word]

  except:
    return vector
  return vector

# 
def give_embedding(x):
  (batch_size, seq_len,temp) = x.shape
  ans = torch.zeros(batch_size, seq_len, 50)
  for i in range(batch_size):
    for j in range(seq_len):
      emb = find_embedding(idx_to_word[x[i][j].item()])
      for k in range(len(emb)):
        ans[i][j][k] = float(emb[k])
  return ans


  

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


/home/user/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [16]:
# 
class ELMO(nn.Module):
    def __init__(self, hidden_size, req_emb_size,input_size,vocb_size):
        super(ELMO, self).__init__()
        self.vocb_size = vocb_size
        self.Bi1 = nn.LSTM(input_size, hidden_size, bidirectional = True, batch_first=True )
        self.Bi2 = nn.LSTM(hidden_size*2, hidden_size, bidirectional = True, batch_first=True )
        self.linear1 = nn.Linear(hidden_size*2, vocb_size)
        
        # keeping weights
        self.w1 =  torch.nn.Parameter(torch.randn(50,req_emb_size))
        self.w2 =  torch.nn.Parameter(torch.randn(2*hidden_size,req_emb_size))
        self.w3 =  torch.nn.Parameter(torch.randn(2*hidden_size,req_emb_size))
        
        

    # 
    def forward(self, x):
        glove_emb = give_embedding(x)
        
        Bi1out,_ = self.Bi1(glove_emb)
        Bi2out,_ = self.Bi2(Bi1out)
        last_word = Bi2out[:,-1,:] # last word of the sentence
        out = self.linear1(last_word)
        out = F.softmax(out, dim=1)

        return out

    # will give sentence embedding
    def give_elmo(self,x):
      glove_emb = give_embedding(x)
      Bi1out,_ = self.Bi1(glove_emb)
      Bi2out,_ = self.Bi2(Bi1out)

      t1 = torch.matmul(glove_emb, self.w1)
      t2 = torch.matmul(Bi1out, self.w2)
      t3 = torch.matmul(Bi2out, self.w3)

      temp1 = torch.add(t2, t3)
      temp2 = torch.add(t1, temp1)

      return (temp2) #embedding of the sentence
      





In [17]:
# splitting into batches
batches = [[] for i in range(0,1000)]
import copy
train_elmo_x = []
train_elmo_y = []

for sentence in sentences:
  words = []
  for word in sentence:
    if(len(words)==0):
      words.append(word)
    else:
      batches[len(words)].append((copy.deepcopy(words), word))
      train_elmo_x.append(copy.deepcopy(words))
      train_elmo_y.append(word)
      words.append(word)
batches = [b for b in batches if len(b) > 0]

elmo = ELMO(100,100,50, len(vocb)) # hidden size, required embedding size, glove embedding size, vocab size
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(elmo.parameters(), lr=1e-2)

In [18]:
for epoch in range(3):
  total_loss = 0
  batch_num = 0
  for batch in batches:
    batch_size = len(batch)
    seq_len = len(batch[0][0])
    train_x = torch.zeros(batch_size, seq_len, 1)
    train_y = torch.zeros(batch_size, 1)

    for i, (x,y) in enumerate(batch):
      
      index_form = []
      for word in x:
        index_form.append(word_to_idx[word])
      for j in range(len(index_form)):
        train_x[i][j] = index_form[j]
      train_y[i] = word_to_idx[y]


    out = elmo(train_x)
    

    loss = criterion(out, train_y.squeeze(dim=1).type(torch.LongTensor))
    total_loss +=loss.item()
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    batch_num+=1

  print("EPOCH: " + str(epoch+1) + " LOSS: " + str(total_loss))

  

EPOCH: 1 LOSS: 1482.2344284057617


: 

: 

In [1]:
torch.save(elmo, "/home/user/Desktop/elmo/anlp-assgn2-data/elmo-v1.pt")

NameError: name 'torch' is not defined

## classification

In [ ]:
elmo = torch.load("/home/user/Desktop/elmo/anlp-assgn2-data/elmo-v1.pt")

def generate_elmo(sentence):
  batch_size = 1
  seq_len = len(sentence)
  to_feed = torch.zeros(batch_size, seq_len, 1)
    
  for j in range(seq_len):
    to_feed[0][j] = sentence[j]

  
  return elmo.give_elmo(to_feed)

  

In [ ]:
class DOWN(nn.Module):
    def __init__(self, emb_size):
        super(DOWN, self).__init__()
        self.linear1 = nn.Linear(emb_size, 300)
        self.linear2 = nn.Linear(300, 5)

      

    def forward(self, x):
        elmo_emb = generate_elmo(x).squeeze(dim=0)
        elmo_emb = torch.flatten(elmo_emb)
        
        
        
        
        out = self.linear1(elmo_emb)
        out = F.relu(out)

        out = self.linear2(out)
        out = F.softmax(out, dim=0)

        return out



In [ ]:
down = DOWN(3000)
criterion1 = nn.CrossEntropyLoss()
optimizer1 = optim.SGD(down.parameters(), lr=1e-2)

In [ ]:
# test set
x=[]
y=[]
#iterate over the rows of the dataframe to select the text and label
limit = 30
for index, row in new_data.iterrows():
    
    text = row['text'].replace('.',' ')
    words = text.split()[:limit]
    
    x.append(words)
    y.append(int(row['label']))

# for each x if the length is less than limit, add padding
for i in range(len(x)):
    if len(x[i]) < limit:
        for j in range(limit-len(x[i])):
            x[i].append('pad')

In [ ]:
for epoch in range(5):
  
  req = 0
  for i in range(len(x)):
    temp_x = torch.zeros(30)
    for j,word in enumerate(x[i]):
      temp_x[j]= (word_to_idx[word])
      

    
    out = down(temp_x)
    amma = out.argmax().item()
    if(amma == int(y[i])):
      req+=1
    temp_y = torch.zeros(1)
    temp_y[0] = int(y[i])
    

    loss = criterion1(out, temp_y.squeeze(dim=0).type(torch.LongTensor))
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()

  print("EPOCH: " + str(epoch) + "Accuracy: " + str(req/len(x)))
  


  
  



EPOCH: 0Accuracy: 0.208
EPOCH: 1Accuracy: 0.199
EPOCH: 2Accuracy: 0.198
EPOCH: 3Accuracy: 0.197
EPOCH: 4Accuracy: 0.199


In [ ]:
from sklearn import metrics


total = len(x)
req = 0

a = []
b=[]
for i in range(len(x)):
  temp_x = torch.zeros(30)
  for j,word in enumerate(x[i]):
    temp_x[j]= (word_to_idx[word])
    

  
  out = down(temp_x)
  out = out.argmax().item()
  if(out == int(y[i])):
    req+=1
  a.append(out)
  b.append(int(y[i]))




print("Accuracy is : " + str(req*100/total))

a = np.array(a)
b = np.array(b)
metrics.confusion_matrix(b, a)


Accuracy is : 20.1


array([[  0,   0,   1,   1, 185],
       [  0,   0,   0,   1, 208],
       [  0,   0,   2,   0, 211],
       [  0,   0,   1,   3, 190],
       [  0,   0,   1,   0, 196]])

In [ ]:
from sklearn import metrics
print(metrics.classification_report(b, a))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.00      0.00      0.00       209
           2       0.40      0.01      0.02       213
           3       0.60      0.02      0.03       194
           4       0.20      0.99      0.33       197

    accuracy                           0.20      1000
   macro avg       0.24      0.20      0.08      1000
weighted avg       0.24      0.20      0.07      1000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
